# Linear Regression Code Along

Basically what I do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [3]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [4]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True,
                      header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.describe().show()

+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|summary|            Email|             Address|     Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|  count|              500|                 500|        500|               500|               500|               500|                 500|                500|
|   mean|             null|                null|       null| 33.05319351819619|12.052487937166134| 37.06044542094859|   3.533461555915055|  499.3140382585909|
| stddev|             null|                null|       null|0.9925631110845354|0.9942156084725424|1.0104889067564033|  0.9992775024112585|   79.3147815497068|
|    min|aaron04@yahoo.com|0001 Mack MillNor..

In [7]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [8]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


## Setting Up DataFrame for Machine Learning 


A few things we need to do before Spark can accept the data!

- It needs to be in the form of two columns
("label","features")

- Import VectorAssembler and Vectors

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [11]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App',
                                       'Time on Website','Length of Membership','Yearly Amount Spent'],
                            outputCol='features')

In [12]:
output = assembler.transform(data)

In [13]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [14]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [15]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [16]:
final_data = output.select('features', 'Yearly Amount Spent')

In [17]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [18]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [19]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                355|
|   mean|  498.4920266183519|
| stddev|  78.22329274120919|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [20]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                145|
|   mean|  501.3265495157271|
| stddev|  82.16744966964666|
|    min|   266.086340948469|
|    max|  725.5848140556806|
+-------+-------------------+



In [21]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [22]:
lr_model = lr.fit(train_data)

In [23]:
test_results = lr_model.evaluate(test_data)

In [24]:
test_results.r2

1.0

In [25]:
test_results.rootMeanSquaredError

2.3640679527554053e-13

In [26]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|2.273736754432320...|
|1.136868377216160...|
|3.410605131648481...|
|-2.27373675443232...|
|-5.68434188608080...|
|-5.68434188608080...|
|1.136868377216160...|
|1.705302565824240...|
|                 0.0|
|-5.68434188608080...|
|1.136868377216160...|
|5.684341886080801...|
|5.684341886080801...|
|-1.70530256582424...|
|1.705302565824240...|
|2.842170943040400...|
|-3.41060513164848...|
|2.842170943040400...|
|-1.13686837721616...|
|-3.41060513164848...|
+--------------------+
only showing top 20 rows



In [27]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [28]:
test_results.r2

1.0

In [29]:
unlabeled_data = test_data.select('features')

In [30]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.4925366965402...|
|[30.8162006488763...|
|[31.1695067987115...|
|[31.2834474760581...|
|[31.3895854806643...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.5761319713222...|
|[31.7366356860502...|
|[31.8093003166791...|
|[31.8124825597242...|
|[31.8186165667690...|
|[31.8209982016720...|
|[31.8279790554652...|
|[31.9048571310136...|
|[31.9096268275227...|
|[31.9365486184489...|
|[31.9480174211613...|
|[32.0180740106320...|
+--------------------+
only showing top 20 rows



In [31]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 319.9288698031934|
|[30.4925366965402...| 282.4712457199144|
|[30.8162006488763...|266.08634094846866|
|[31.1695067987115...|427.35653080229304|
|[31.2834474760581...|  591.781089425668|
|[31.3895854806643...|410.06961105998295|
|[31.5257524169682...|443.96562680988177|
|[31.5261978982398...|409.09452619233764|
|[31.5761319713222...| 541.2265839893283|
|[31.7366356860502...|  496.933446255532|
|[31.8093003166791...|  536.771899362841|
|[31.8124825597242...|392.81034498379717|
|[31.8186165667690...| 446.4186733701356|
|[31.8209982016720...| 424.6752810132135|
|[31.8279790554652...|440.00274754694135|
|[31.9048571310136...|473.94985742281585|
|[31.9096268275227...| 563.4460356732395|
|[31.9365486184489...| 427.1993848953279|
|[31.9480174211613...| 461.9208768928979|
|[32.0180740106320...|357.78311074531564|
+--------------------+------------

In [32]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 2.3640679527554053e-13
MSE: 5.588817285245134e-26
